In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import seaborn as sns
from sklearn.metrics import confusion_matrix

In [ ]:
df = pd.read_csv('https://drive.google.com/uc?export=download&id=1-d9-P77e1yXzGwGO2nDqq8wHDRjuZhqy')
df

,_TOTINDA,_EXTETH3,_DENVST3,_LTASTH1,_CASTHM1,_DRDXAR2,_RACEG22,_AGE65YR,WTKG3,_INCOMG1,...,RaceTreat2-Worse than other,Health-Fair or Poor,Health-Good or Better,Health not good:0day,Health not good:1-13day,Health not good:14+day,mental not good:0day,mental not good:1-13day,mental not good:14+day,Class
0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,63.500000,7.0,...,0,0,1,1,0,0,1,0,0,1
1,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,77.110000,5.0,...,0,0,1,1,0,0,1,0,0,0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,83.835752,6.0,...,0,0,1,0,1,0,0,0,0,1
3,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,102.060000,4.0,...,0,1,0,0,1,0,0,1,0,0
4,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,77.110000,4.0,...,0,0,1,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51094,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,97.980000,1.0,...,0,1,0,0,1,0,0,0,1,0
51095,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,83.835752,4.0,...,0,0,1,1,0,0,1,0,0,0
51096,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,83.910000,3.0,...,0,1,0,0,0,1,0,0,1,0
51097,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,83.910000,3.0,...,0,0,1,0,1,0,1,0,0,0


In [ ]:
num_cols = ['WTKG3', '_DRNKWK2', 'HHADULT', 'PHYSHLTH', 'MENTHLTH', 'POORHLTH']
cat_cols = [col for col in df.columns if col not in num_cols][:-1]
cat_cols

['_TOTINDA',
 '_EXTETH3',
 '_DENVST3',
 '_LTASTH1',
 '_CASTHM1',
 '_DRDXAR2',
 '_RACEG22',
 '_AGE65YR',
 '_INCOMG1',
 '_RFMAM22',
 '_RFSMOK3',
 'DRNKANY6',
 '_AIDTST4',
 '_ASTHMS1__1.0',
 '_ASTHMS1__2.0',
 '_ASTHMS1__3.0',
 '_ASTHMS1__9.0',
 '_EDUCAG__1.0',
 '_EDUCAG__2.0',
 '_EDUCAG__3.0',
 '_EDUCAG__4.0',
 '_EDUCAG__9.0',
 '_SMOKER3__1.0',
 '_SMOKER3__2.0',
 '_SMOKER3__3.0',
 '_SMOKER3__4.0',
 '_SMOKER3__9.0',
 'DEAF',
 'BLIND',
 'DECIDE',
 'DIFFWALK',
 'DIFFDRES',
 'DIFFALON',
 'LCSCTSC1',
 'HIVTST7',
 'SEXVAR',
 'PRIMINSR',
 'MEDCOST1',
 'EXERANY2',
 'LASTDEN4',
 'RMVTETH4',
 'CVDSTRK3',
 'ASTHMA3',
 'CHCCOPD3',
 'ADDEPEV3',
 'HAVARTH4',
 'EDUCA',
 'RENTHOM1',
 'EMPLOY1',
 'INCOME3',
 'EMTSUPRT',
 'SDHISOLT',
 'SDHEMPLY',
 'FOODSTMP',
 'SDHFOOD1',
 'SDHBILLS',
 'SDHUTILS',
 'Dissatisfied',
 'Satisfied',
 'Very Dissatisfied',
 'Very satisfied',
 'SDHSTRE1',
 'RaceTreat-Better',
 'RaceTreat-Better or Worse',
 'RaceTreat-Only Encounter Same',
 'RaceTreat-Same',
 'RaceTreat-Worse',
 'R

In [ ]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df[cat_cols].iloc[:,:-1].values, df.iloc[:,-1].values, test_size=0.2, random_state=42)


In [ ]:
def gini_impurity_gain(df, target_col):
    total_samples = len(df)
    initial_gini = 1 - (df[target_col].value_counts(normalize=True) ** 2).sum()

    gain_dict = {}

    for col in df.columns:
        if col != target_col:
            weighted_gini = 0
            for value in df[col].unique():
                subset = df[df[col] == value]
                subset_size = len(subset)
                value_gini = 1 - (subset[target_col].value_counts(normalize=True) ** 2).sum()
                weighted_gini += (subset_size / total_samples) * value_gini
            gini_impurity_gain = initial_gini - weighted_gini
            gain_dict[col] = gini_impurity_gain

    sorted_gains = sorted(gain_dict.items(), key=lambda x: x[1], reverse=True)

    print(f"Initial Gini Impurity: {initial_gini:.4f}\n")

    for col, gain in sorted_gains:
        print(f"Gini Impurity Gain for {col}: {gain:.4f}")

In [32]:
gini_impurity_gain(df, 'Class')

Initial Gini Impurity: 0.1973

Gini Impurity Gain for DECIDE: 0.0220
Gini Impurity Gain for POORHLTH: 0.0083
Gini Impurity Gain for MENTHLTH: 0.0081
Gini Impurity Gain for SDHSTRE1: 0.0067
Gini Impurity Gain for PHYSHLTH: 0.0066
Gini Impurity Gain for DIFFALON: 0.0064
Gini Impurity Gain for Health-Good or Better: 0.0063
Gini Impurity Gain for Health-Fair or Poor: 0.0063
Gini Impurity Gain for mental not good:14+day: 0.0061
Gini Impurity Gain for ADDEPEV3: 0.0057
Gini Impurity Gain for DIFFWALK: 0.0055
Gini Impurity Gain for mental not good:0day: 0.0055
Gini Impurity Gain for Health not good:14+day: 0.0051
Gini Impurity Gain for EMPLOY1: 0.0048
Gini Impurity Gain for SDHISOLT: 0.0048
Gini Impurity Gain for Health not good:0day: 0.0041
Gini Impurity Gain for DIFFDRES: 0.0039
Gini Impurity Gain for EMTSUPRT: 0.0038
Gini Impurity Gain for SDHFOOD1: 0.0031
Gini Impurity Gain for INCOME3: 0.0029
Gini Impurity Gain for Dissatisfied: 0.0029
Gini Impurity Gain for _INCOMG1: 0.0028
Gini Impurity

In [ ]:
def entropy(series, target):
    levels = series.unique()
    ents = []
    for level in levels:
      level_sub = series[series == level].index
      p = target[level_sub].value_counts(normalize=True)
      ent = -(p * np.log2(p)).sum()
      ents.append(ent)
    w = series.value_counts(normalize=True)
    return np.dot(ents, w)

def entropy_of_feature(df, feature):
    ent = entropy(df[feature], df.Class)
    return ent

def entropy_of_all_features(df):
    ent_dict = {}
    for col in df.columns:
        ent_dict[col] = entropy_of_feature(df, col)

    sorted_entropies = sorted(ent_dict.items(), key=lambda x: x[1])

    for col, ent in sorted_entropies:
        print(f"Entropy of {col}: {ent:.7f}")

In [ ]:
entropy_of_all_features(df)

Entropy of Class: 0.0000000
Entropy of Very satisfied: 0.5428733
Entropy of DECIDE: 0.5432547
Entropy of _DRNKWK2: 0.5614632
Entropy of _INCOMG1: 0.5651468
Entropy of POORHLTH: 0.5742899
Entropy of MENTHLTH: 0.5761201
Entropy of PHYSHLTH: 0.5761986
Entropy of DIFFALON: 0.5986264
Entropy of Health-Good or Better: 0.5991204
Entropy of Health-Fair or Poor: 0.5992753
Entropy of mental not good:0day: 0.6031132
Entropy of SDHSTRE1: 0.6040705
Entropy of DIFFWALK: 0.6041315
Entropy of ADDEPEV3: 0.6041466
Entropy of mental not good:14+day: 0.6043709
Entropy of Health not good:14+day: 0.6072308
Entropy of WTKG3: 0.6085256
Entropy of _EDUCAG__4.0: 0.6089150
Entropy of SDHISOLT: 0.6089626
Entropy of Health not good:0day: 0.6090801
Entropy of DIFFDRES: 0.6153818
Entropy of SDHBILLS: 0.6235776
Entropy of Dissatisfied: 0.6240143
Entropy of _DRDXAR2: 0.6266938
Entropy of HAVARTH4: 0.6266938
Entropy of BLIND: 0.6267849
Entropy of _TOTINDA: 0.6273100
Entropy of EXERANY2: 0.6273544
Entropy of CHCCOPD3: 0

In [ ]:
# series = df.INCOME3
# target = df.Class
# series.value_counts(normalize=True)

# levels = series.unique()
# ents = []
# for level in levels:
#   level_sub = series[series == level].index
#   p = target[level_sub].value_counts(normalize=True)
#   ent = -(p * np.log2(p)).sum()
#   ents.append(ent)
# w = series.value_counts(normalize=True)